In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date
from unidecode import unidecode
from tqdm.notebook import tqdm
from glob import glob

BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Nuevos atributos

A los anteriores, se agrega sub tipo de institución: 
- CFT: Estatal / Privado
- IP: Presencia nacional / Privado

# Raw data

In [2]:
df = pd.read_csv(f"{BASE}work_data/matriz_desercion_4.csv.gz")
print(df.columns)
df.head()

/tmp/ipykernel_7503/621277867.py:1: DtypeWarning: Columns (17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{BASE}work_data/matriz_desercion_4.csv.gz")


Index(['cat_periodo', 'codigo_unico', 'mrun', 'gen_alu', 'fec_nac_alu',
       'anio_ing_carr_ori', 'tipo_inst_1', 'jornada', 'dur_estudio_carr',
       'dur_proceso_tit', 'region_sede', 'formato_valores', 'valor_arancel',
       'area_conocimiento', 'acreditada_carr', 'acreditada_inst', 'desertor_1',
       'region_colegio', 'tipo_ensenanza_colegio', 'prom_notas_media',
       'dependencia_colegio', 'fscu', 'gratuidad', 'beca'],
      dtype='object')


,cat_periodo,codigo_unico,mrun,gen_alu,fec_nac_alu,anio_ing_carr_ori,tipo_inst_1,jornada,dur_estudio_carr,dur_proceso_tit,...,acreditada_carr,acreditada_inst,desertor_1,region_colegio,tipo_ensenanza_colegio,prom_notas_media,dependencia_colegio,fscu,gratuidad,beca
0,2015,I498S6C132J2V1,37.0,1,197808,2015,Centros de Formación Técnica,Vespertino,5,1.0,...,NO ACREDITADA,ACREDITADA,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,2015,I111S14C309J2V1,118.0,1,199102,2015,Institutos Profesionales,Vespertino,5,1.0,...,ACREDITADA,ACREDITADA,0.0,13,310.0,4.975,3.0,0.0,0.0,0.0
2,2015,I106S3C64J4V1,253.0,1,198404,2015,Institutos Profesionales,A Distancia,5,1.0,...,NO ACREDITADA,NO ACREDITADA,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,2015,I221S1C57J1V1,370.0,2,198910,2015,Centros de Formación Técnica,Diurno,5,1.0,...,ACREDITADA,ACREDITADA,1.0,13,310.0,5.100,3.0,0.0,0.0,1.0
4,2015,I176S8C59J2V1,533.0,2,198309,2015,Institutos Profesionales,Vespertino,5,1.0,...,ACREDITADA,NO ACREDITADA,0.0,4,510.0,5.200,2.0,0.0,0.0,0.0


## Selección periodo
- `cat_periodo` entre 2015 y 2019 inclusive. 

In [3]:
df = df[df["cat_periodo"] < 2020]

In [4]:
df_mrun = df[["cat_periodo", "mrun"]].drop_duplicates()
df_mrun.shape

(1797138, 2)

In [5]:
df_mrun.to_csv(f"{BASE}work_data/mrun_anio.csv", index=False)

# Nulos

In [6]:
df["fec_nac_alu"] = df["fec_nac_alu"].replace([190001], [np.nan])
df["anio_ing_carr_ori"] = df["anio_ing_carr_ori"].replace([9995], [np.nan])
df["acreditada_carr"] = df["acreditada_carr"].str.strip().replace(["", "SIN INFORMACION"], [np.nan, np.nan])
df["acreditada_inst"] = df["acreditada_inst"].str.strip().replace([""], [np.nan])
print(df.shape)
df = df.dropna()
print(df.shape)

(1802577, 24)
(1560671, 24)


# Pre procesamiento variables dependientes datos desagregados
## Diferencia año ingreso y último año de matrícula

In [7]:
g = df.groupby(["mrun", "codigo_unico"]).agg({
    "cat_periodo": "max",
    "anio_ing_carr_ori": "min"
}).reset_index()
g["dif_ing_ult_mat"] = g["cat_periodo"] - g["anio_ing_carr_ori"]
g.head()

,mrun,codigo_unico,cat_periodo,anio_ing_carr_ori,dif_ing_ult_mat
0,19.0,I536S0C39J1V1,2019,2019,0
1,118.0,I111S14C309J2V1,2015,2015,0
2,118.0,I221S1C113J2V1,2017,2015,2
3,196.0,I280S8C380J1V1,2019,2017,2
4,238.0,I143S15C614J1V1,2017,2016,1


In [8]:
if "dif_ing_ult_mat" not in df.columns:
    print(df.shape)
    df = pd.merge(
        df,
        g[["mrun", "codigo_unico", "dif_ing_ult_mat"]],
        how="left"
    )
    print(df.shape)

(1560671, 24)
(1560671, 25)


In [9]:
df["dif_ing_ult_mat"].value_counts("%") * 100

2    31.432698
1    28.825102
0    23.873257
3    12.011949
4     3.856995
Name: dif_ing_ult_mat, dtype: float64

## Edad

In [10]:
# Se pisa año de ingreso por el minimo
print(df.shape)
df = pd.merge(
    df.drop(columns=["anio_ing_carr_ori"]),
    g[["mrun", "codigo_unico", "anio_ing_carr_ori"]],
    how="left"
)
df.shape

(1560671, 25)


(1560671, 25)

In [11]:
df["fecha_n"] = pd.to_datetime([
    date(int(str(d)[:4]), int(str(d)[-2:]), 15)
    for d in df["fec_nac_alu"].astype(int)
])

df["Edad"] = [
    date(y, 1, 1).year - fn.year - (
        (date(y, 1, 1).month, date(y, 1, 1).day) < (fn.month, fn.day)
    )
    for fn, y in zip(df["fecha_n"], df["anio_ing_carr_ori"])
]
df.shape

(1560671, 27)

### Valor arancel

In [12]:
valores_uf_dic_anio_anterior = {
    2014: 24627.10,
    2015: 25629.09,
    2016: 26347.98,
    2017: 26798.14,
    2018: 27565.79,
}

In [13]:
df["valor_arancel"] = [
    int(
        valores_uf_dic_anio_anterior[a] * va
    ) if fv == "Monto en UF" else va
    for a, fv, va in zip(
        df["cat_periodo"], df["formato_valores"], df["valor_arancel"]
    )
]

In [14]:
g = df.groupby(["mrun", "codigo_unico"]).agg({
    "valor_arancel": "mean",
}).reset_index()

print(df.shape)
df = pd.merge(
    df.drop(columns=["valor_arancel"]),
    g[["mrun", "codigo_unico", "valor_arancel"]],
    how="left"
)
print(df.shape)

(1560671, 27)
(1560671, 27)


# Registros únicos

- Se crea 1 registro por combinación `mrun` - `codigo_unico`.

In [15]:
for c in ["cat_periodo", "anio_ing_carr_ori", "fecha_n", "fec_nac_alu"]:
    if c in df.columns:
        df = df.drop(columns=[c])
        
df.shape

(1560671, 23)

In [16]:
# Se mantiene el registro donde esté como desertor
t = df.drop_duplicates().sort_values(by=["mrun", "codigo_unico", "desertor_1"], ascending=[False, False, False]).drop_duplicates(subset=["mrun", "codigo_unico"], keep="first").copy()
t.shape

(885780, 23)

In [17]:
# No debe haber mrun-codigo_unico repetidos
assert t[["mrun", "codigo_unico"]].value_counts().reset_index()[0].max() == 1

In [18]:
# Se revisa dif
t["dif_ing_ult_mat"].value_counts("%") * 100

0    42.062702
1    27.811985
2    21.018086
3     6.998578
4     2.108650
Name: dif_ing_ult_mat, dtype: float64

In [19]:
print("Desertores")
print(t[t["desertor_1"]==1]["dif_ing_ult_mat"].value_counts("%") * 100)
print("\nNo desetores")
print(t[t["desertor_1"]==0]["dif_ing_ult_mat"].value_counts("%") * 100)

Desertores
0    58.733522
1    24.369768
2    11.744449
3     4.046829
4     1.105432
Name: dif_ing_ult_mat, dtype: float64

No desetores
0    36.789405
1    28.900824
2    23.951513
3     7.932271
4     2.425987
Name: dif_ing_ult_mat, dtype: float64


In [20]:
t["desertor_1"].value_counts()

0.0    672922
1.0    212858
Name: desertor_1, dtype: int64

# Pre procesamiento resto de variables
## Tipo enseñanza

In [21]:
omit_dep = [
    'LOS ANDES', 'SANTIAGO ORIENTE', 'VALPARAISO', 'SANTIAGO PONIENTE',
    'COLCHAGUA', 'VALPARAÍSO', 'CAUTÍN NORTE', 'ANTOFAGASTA - TOCOPILLA'
]

t = t[~t["dependencia_colegio"].isin(omit_dep)]
t.shape

(885579, 23)

In [22]:
t["dependencia_colegio"] = t["dependencia_colegio"].astype(int)
t["dependencia_colegio"].unique()

array([3, 2, 1, 6, 5, 4])

In [23]:
t["dependencia_colegio"] = t["dependencia_colegio"].map({
    1: 'Municipal',
    2: 'Municipal',
    3: 'Particular Subvencionado',
    4: 'Particular Pagado',
    5: 'Otro',
    6: 'Otro'
})
t["dependencia_colegio"].value_counts()

Particular Subvencionado    444148
Municipal                   361086
Otro                         50947
Particular Pagado            29398
Name: dependencia_colegio, dtype: int64

## Dependencia Colegio

In [24]:
t["tipo_ensenanza_colegio"] = t["tipo_ensenanza_colegio"].map({
    0: "no_aplica",
    10: "Parvularia",
    110: "Basica",
    160: "Basica",
    161: "Basica",
    163: "Carceles",
    165: "Basica",
    167: "Basica",
    211: "Especial",
    212: "Especial",
    213: "Especial",
    214: "Especial",
    215: "Especial",
    216: "Especial",
    217: "Especial",
    299: "PIE",
    310: "HC",
    360: "HC",
    361: "HC",
    362: "Carceles",
    363: "HC",
    410: "TP Comercial",
    460: "TP Comercial",
    461: "TP Comercial",
    463: "TP Comercial",
    510: "TP Industrial",
    560: "TP Industrial",
    561: "TP Industrial",
    563: "TP Industrial",
    610: "TP Tecnica",
    660: "TP Tecnica",
    661: "TP Tecnica",
    663: "TP Tecnica",
    710: "Otro",
    760: "Otro",
    761: "Otro",
    763: "Otro",
    810: "Otro",
    860: "Otro",
    863: "Otro",
    910: "Otro",
    963: "Otro"                     
})

t["tipo_ensenanza_colegio"].value_counts()

HC               500442
TP Comercial     143680
TP Industrial    140212
TP Tecnica        82524
Otro              18721
Name: tipo_ensenanza_colegio, dtype: int64

## Región colegio

In [25]:
t["region_colegio"] = t["region_colegio"].astype(int)
t["region_colegio"].unique()

array([13,  7,  2,  6,  9,  5, 10, 16,  4,  1, 14, 15, 12,  8, 11,  3])

In [26]:
t["region_colegio"] = t["region_colegio"].map({
    1: "Tarapacá",
    2: "Antofagasta",
    3: "Atacama",
    4: "Coquimbo",
    5: "Valparaíso",
    6: "Lib. Gral B. O'Higgins",
    7: "Maule",
    8: "Biobío",
    9: "La Araucanía",
    10: "Los Lagos",
    11: "Aysén",
    12: "Magallanes",
    13: "Metropolitana",
    14: "Los Ríos",
    15: "Arica y Parinacota",
    16: "Ñuble"
})
t["region_colegio"].value_counts()

Metropolitana             354424
Valparaíso                 91111
Biobío                     80666
Maule                      51746
Lib. Gral B. O'Higgins     49053
La Araucanía               46539
Coquimbo                   40650
Los Lagos                  39955
Antofagasta                32651
Ñuble                      21082
Los Ríos                   19441
Atacama                    15566
Tarapacá                   13626
Arica y Parinacota         13157
Magallanes                  9534
Aysén                       6378
Name: region_colegio, dtype: int64

In [27]:
t["misma_region"] = np.where(t["region_colegio"]==t["region_sede"], 1, 0)
t["misma_region"].value_counts("%") * 100

1    85.803751
0    14.196249
Name: misma_region, dtype: float64

## Genero, desertor, beneficios

In [28]:
t["gen_alu"] = t["gen_alu"].replace([1, 2], ["hombre", "mujer"])
t["desertor_1"] = t["desertor_1"].astype(int)
t["fscu"] = t["fscu"].astype(int)
t["gratuidad"] = t["gratuidad"].astype(int)
t["beca"] = t["beca"].astype(int)
t["gen_alu"].value_counts()

hombre    443207
mujer     442372
Name: gen_alu, dtype: int64

## Duraciones

In [29]:
t["dur_estudio_carr"] = t["dur_estudio_carr"].astype(int)
t["dur_proceso_tit"] = t["dur_proceso_tit"].astype(int)

## Instituciones

In [30]:
def iter_df_read(file, col_name, condition_list, chunksize=1000, **kwargs):
    iter_csv = pd.read_csv(file, iterator=True, chunksize=chunksize, **kwargs)
    return pd.concat([chunk[chunk[col_name].isin(condition_list)] for chunk in iter_csv])

In [31]:
cols = ['tipo_inst_1', 'codigo_unico', 'nomb_inst']
tipo_inst = ['Centros de Formación Técnica', 'Institutos Profesionales']

instituciones = pd.concat([
    iter_df_read(f, "tipo_inst_1", tipo_inst, sep=";", usecols=cols)
    for f in tqdm(glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"), total=8)
]).drop_duplicates().reset_index(drop=True)

  0%|          | 0/8 [00:00<?, ?it/s]

In [32]:
assert instituciones.groupby(["codigo_unico"]).agg({"nomb_inst": pd.Series.nunique}).reset_index()["nomb_inst"].max() == 1

In [33]:
if "nomb_inst" not in t.columns:     
    t_inicial = t.shape[0]
    t = pd.merge(
        t,
        instituciones,
        how="left"
    )

    assert t.shape[0] == t_inicial
    
    cft_estatal = [
        'CFT DE LA REGION DE ANTOFAGASTA',
        'CFT DE LA REGION DE ARICA Y PARINACOTA',
        'CFT DE LA REGION DE ATACAMA',
        'CFT DE LA REGION DE AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO',
        'CFT DE LA REGION DE COQUIMBO',
        'CFT DE LA REGION DE LA ARAUCANIA',
        'CFT DE LA REGION DE LOS LAGOS',
        'CFT DE LA REGION DE LOS RIOS',
        'CFT DE LA REGION DE MAGALLANES Y ANTARTICA CHILENA',
        'CFT DE LA REGION DE TARAPACA',
        'CFT DE LA REGION DE VALPARAISO',
        'CFT DE LA REGION DEL BIO BIO',
        "CFT DE LA REGION DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS",
        'CFT DE LA REGION DEL MAULE',
        'CFT DE LA REGION METROPOLITANA DE SANTIAGO'
    ]
    
    ip_nac = [
        'IP AIEP',
        'IP DE CHILE',
        'IP DUOC UC',
        'IP INACAP',
        'IP IPG',
        'IP LATINOAMERICANO DE COMERCIO EXTERIOR - IPLACEX',
        'IP SANTO TOMAS',
    ]

    t["sub_tipo_inst"] = np.where(
        t["nomb_inst"].isin(cft_estatal),
        "cft_estatal",
        np.where(
            t["tipo_inst_1"] == "Centros de Formación Técnica",
            "cft_privado",
            np.where(
                t["nomb_inst"].isin(ip_nac),
                "ip_privado_precencia_nacional",
                "ip_privado"
            )
        )
    )

    assert t.shape[0] == t_inicial
    
t["sub_tipo_inst"].value_counts(dropna=False)

ip_privado_precencia_nacional    479666
cft_privado                      281751
ip_privado                       123265
cft_estatal                         897
Name: sub_tipo_inst, dtype: int64

# Sacar cols auxiliares

In [34]:
t_aux = t.copy().drop(columns=[
    "region_colegio", "nomb_inst", "formato_valores"
]).reset_index(drop=True)

dummy_aux = pd.get_dummies(
    t_aux.drop(columns=["codigo_unico", "mrun"])
).reset_index(drop=True)

dummy_aux = pd.concat([
    t_aux[["mrun", "codigo_unico"]],
    dummy_aux
], axis=1)

dummy_aux.columns = [
    unidecode(c.replace(" ", "_").replace(".", "").replace("'", "").lower())
    for c in dummy_aux.columns
]
dummy_aux.to_csv(f"{BASE}work_data/matriz_desercion_4_dummy_con_id.csv.gz", index=False, compression="gzip")

In [35]:
t = t.drop(columns=["codigo_unico", "mrun", "region_colegio", "nomb_inst", "formato_valores"])

# No dummy

In [36]:
t.head()

,gen_alu,tipo_inst_1,jornada,dur_estudio_carr,dur_proceso_tit,region_sede,area_conocimiento,acreditada_carr,acreditada_inst,desertor_1,...,prom_notas_media,dependencia_colegio,fscu,gratuidad,beca,dif_ing_ult_mat,Edad,valor_arancel,misma_region,sub_tipo_inst
0,mujer,Centros de Formación Técnica,Vespertino,4,1,Metropolitana,Administración y Comercio,NO ACREDITADA,ACREDITADA,1,...,6.4,Particular Subvencionado,0,0,0,0,35,1390000.0,1,cft_privado
1,hombre,Institutos Profesionales,Vespertino,5,1,Metropolitana,Tecnología,ACREDITADA,ACREDITADA,1,...,5.5,Particular Subvencionado,0,0,1,0,31,1850000.0,1,ip_privado_precencia_nacional
2,hombre,Centros de Formación Técnica,Vespertino,6,1,Maule,Tecnología,NO ACREDITADA,NO ACREDITADA,0,...,4.7,Particular Subvencionado,0,0,0,0,29,803000.0,1,cft_privado
3,hombre,Institutos Profesionales,Vespertino,5,1,Metropolitana,Tecnología,NO ACREDITADA,ACREDITADA,0,...,6.0,Municipal,0,0,0,0,29,1690000.0,1,ip_privado_precencia_nacional
4,mujer,Institutos Profesionales,Diurno,5,1,Metropolitana,Derecho,NO ACREDITADA,ACREDITADA,0,...,6.4,Particular Subvencionado,0,0,0,0,16,1480000.0,1,ip_privado_precencia_nacional


In [37]:
t.to_csv(f"{BASE}work_data/matriz_desercion_4_no_dummy.csv.gz", index=False, compression="gzip")

# Dummys

In [38]:
t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 885579 entries, 0 to 885578
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   gen_alu                 885579 non-null  object 
 1   tipo_inst_1             885579 non-null  object 
 2   jornada                 885579 non-null  object 
 3   dur_estudio_carr        885579 non-null  int64  
 4   dur_proceso_tit         885579 non-null  int64  
 5   region_sede             885579 non-null  object 
 6   area_conocimiento       885579 non-null  object 
 7   acreditada_carr         885579 non-null  object 
 8   acreditada_inst         885579 non-null  object 
 9   desertor_1              885579 non-null  int64  
 10  tipo_ensenanza_colegio  885579 non-null  object 
 11  prom_notas_media        885579 non-null  float64
 12  dependencia_colegio     885579 non-null  object 
 13  fscu                    885579 non-null  int64  
 14  gratuidad           

In [39]:
dummy = pd.get_dummies(t, drop_first=True)
dummy.head()

,dur_estudio_carr,dur_proceso_tit,desertor_1,prom_notas_media,fscu,gratuidad,beca,dif_ing_ult_mat,Edad,valor_arancel,...,tipo_ensenanza_colegio_Otro,tipo_ensenanza_colegio_TP Comercial,tipo_ensenanza_colegio_TP Industrial,tipo_ensenanza_colegio_TP Tecnica,dependencia_colegio_Otro,dependencia_colegio_Particular Pagado,dependencia_colegio_Particular Subvencionado,sub_tipo_inst_cft_privado,sub_tipo_inst_ip_privado,sub_tipo_inst_ip_privado_precencia_nacional
0,4,1,1,6.4,0,0,0,0,35,1390000.0,...,0,0,0,0,0,0,1,1,0,0
1,5,1,1,5.5,0,0,1,0,31,1850000.0,...,0,0,0,0,0,0,1,0,0,1
2,6,1,0,4.7,0,0,0,0,29,803000.0,...,0,0,0,0,0,0,1,1,0,0
3,5,1,0,6.0,0,0,0,0,29,1690000.0,...,0,0,0,0,0,0,0,0,0,1
4,5,1,0,6.4,0,0,0,0,16,1480000.0,...,0,0,0,0,0,0,1,0,0,1


In [40]:
dummy.columns = [
    unidecode(c.replace(" ", "_").replace(".", "").replace("'", "").lower())
    for c in dummy.columns
]

dummy.columns

Index(['dur_estudio_carr', 'dur_proceso_tit', 'desertor_1', 'prom_notas_media',
       'fscu', 'gratuidad', 'beca', 'dif_ing_ult_mat', 'edad', 'valor_arancel',
       'misma_region', 'gen_alu_mujer', 'tipo_inst_1_institutos_profesionales',
       'jornada_diurno', 'jornada_otro', 'jornada_semipresencial',
       'jornada_vespertino', 'region_sede_arica_y_parinacota',
       'region_sede_atacama', 'region_sede_aysen', 'region_sede_biobio',
       'region_sede_coquimbo', 'region_sede_la_araucania',
       'region_sede_lib_gral_b_ohiggins', 'region_sede_los_lagos',
       'region_sede_los_rios', 'region_sede_magallanes', 'region_sede_maule',
       'region_sede_metropolitana', 'region_sede_tarapaca',
       'region_sede_valparaiso', 'region_sede_nuble',
       'area_conocimiento_agropecuaria',
       'area_conocimiento_arte_y_arquitectura',
       'area_conocimiento_ciencias_basicas',
       'area_conocimiento_ciencias_sociales', 'area_conocimiento_derecho',
       'area_conocimiento_educ

- En género elimina hombre.
- En tipo institución elimina CFT.
- En `region_sede` elimina Antofagasta.
- En `area_conocimiento` elimina Administración y Comercio.
- En ambas acreditaciones elimina Acreditada.
- En `tipo_ensenanza_colegio` elimina HC.
- En `dependencia_colegio` elimina Municipal.

In [41]:
dummy.to_csv(f"{BASE}work_data/matriz_desercion_4_dummy.csv.gz", index=False, compression="gzip")

# Dummy sin drop first

In [42]:
dummy_ohe = pd.get_dummies(t)
dummy_ohe.head()

,dur_estudio_carr,dur_proceso_tit,desertor_1,prom_notas_media,fscu,gratuidad,beca,dif_ing_ult_mat,Edad,valor_arancel,...,tipo_ensenanza_colegio_TP Industrial,tipo_ensenanza_colegio_TP Tecnica,dependencia_colegio_Municipal,dependencia_colegio_Otro,dependencia_colegio_Particular Pagado,dependencia_colegio_Particular Subvencionado,sub_tipo_inst_cft_estatal,sub_tipo_inst_cft_privado,sub_tipo_inst_ip_privado,sub_tipo_inst_ip_privado_precencia_nacional
0,4,1,1,6.4,0,0,0,0,35,1390000.0,...,0,0,0,0,0,1,0,1,0,0
1,5,1,1,5.5,0,0,1,0,31,1850000.0,...,0,0,0,0,0,1,0,0,0,1
2,6,1,0,4.7,0,0,0,0,29,803000.0,...,0,0,0,0,0,1,0,1,0,0
3,5,1,0,6.0,0,0,0,0,29,1690000.0,...,0,0,1,0,0,0,0,0,0,1
4,5,1,0,6.4,0,0,0,0,16,1480000.0,...,0,0,0,0,0,1,0,0,0,1


In [43]:
dummy_ohe.columns = [
    unidecode(c.replace(" ", "_").replace(".", "").replace("'", "").lower())
    for c in dummy_ohe.columns
]

In [44]:
dummy_ohe.to_csv(f"{BASE}work_data/matriz_desercion_4_dummy_ohe.csv.gz", index=False, compression="gzip")